In [2]:
#this program predicts the stock prices using machine learning models

In [3]:
#Importing dependencies

In [6]:
import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [7]:
#get the stock data

In [101]:
df=quandl.get('WIKI/TCS')
#Take a look at the data
print(df.head())

             Open   High    Low  Close      Volume  Ex-Dividend  Split Ratio  \
Date                                                                           
2013-11-01  35.00  37.00  32.10  36.20  14667600.0          0.0          1.0   
2013-11-04  36.80  36.80  34.69  35.35   1586600.0          0.0          1.0   
2013-11-05  35.11  35.50  34.82  35.35    337700.0          0.0          1.0   
2013-11-06  35.61  36.49  35.61  35.90    549600.0          0.0          1.0   
2013-11-07  36.80  36.80  34.12  35.31    883700.0          0.0          1.0   

            Adj. Open  Adj. High  Adj. Low  Adj. Close  Adj. Volume  
Date                                                                 
2013-11-01      35.00      37.00     32.10       36.20   14667600.0  
2013-11-04      36.80      36.80     34.69       35.35    1586600.0  
2013-11-05      35.11      35.50     34.82       35.35     337700.0  
2013-11-06      35.61      36.49     35.61       35.90     549600.0  
2013-11-07      36.

In [103]:
#get the adjusted close price
df=df[['Adj. Close']]
print(df.head())

            Adj. Close
Date                  
2013-11-01       36.20
2013-11-04       35.35
2013-11-05       35.35
2013-11-06       35.90
2013-11-07       35.31


In [104]:
#A variable for predicting n days out into future
forecast_out=30 #next 30 day adj close
#Create a another column (The target or dependent variable) shifted 'n' units up
df['Prediction']=df[['Adj. Close']].shift(-forecast_out)
print('Head')
print(df.head())
print('Tail')
print(df.tail())

Head
            Adj. Close  Prediction
Date                              
2013-11-01       36.20       41.16
2013-11-04       35.35       42.40
2013-11-05       35.35       43.81
2013-11-06       35.90       40.25
2013-11-07       35.31       41.28
Tail
            Adj. Close  Prediction
Date                              
2018-03-21        5.50         NaN
2018-03-22        5.36         NaN
2018-03-23        5.31         NaN
2018-03-26        5.61         NaN
2018-03-27        5.57         NaN


In [105]:
###Create an independent dataset (X) ####

X=np.array(df.drop(['Prediction'],1))
#Remove the last 'n' rows (forecast_out)
X =X[:-forecast_out]
print(X)

[[36.2 ]
 [35.35]
 [35.35]
 ...
 [ 4.86]
 [ 4.67]
 [ 4.77]]


/var/folders/3s/jzblmwmd6bb1z6zxqjy91l8w0000gn/T/ipykernel_47679/738663914.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X=np.array(df.drop(['Prediction'],1))


In [106]:
###Create the dependent dataset (y)###
y=np.array(df['Prediction'])
#Get all of the y values except the last 'n' rows
y=y[:-forecast_out]
print(y)

[41.16 42.4  43.81 ...  5.31  5.61  5.57]


In [107]:
#Split the data into 80% training and 20% testing

x_train, x_test, y_train, y_test=train_test_split(X,y,test_size=0.2)

In [108]:
#create and train the support vector machine (Regressor)
svr_rbf=SVR(kernel='rbf',C=1e3,gamma=0.1)
svr_rbf.fit(x_train,y_train)

SVR(C=1000.0, gamma=0.1)

In [109]:
#Testing Model: Score returns the coefficient of determination of the prediction
#The best possible score is 1.0
svm_confidence=svr_rbf.score(x_test,y_test)
print('svm confidence: ;' ,svm_confidence)

svm confidence: ; 0.9541020120472276


In [110]:
#create and train the linear regression model
lr = LinearRegression()
# Train the model
lr.fit(x_train, y_train)


LinearRegression()

In [111]:
#Testing Model: Score returns the coefficient of determination of the prediction
#The best possible score is 1.0
lr_confidence=lr.score(x_test,y_test)
print('LR confidence: ;' ,lr_confidence)

LR confidence: ; 0.9493381474456675


In [112]:
x_forecast=np.array(df.drop(['Prediction'],1))[-forecast_out:]
print(x_forecast)


[[5.  ]
 [5.09]
 [5.06]
 [4.99]
 [4.85]
 [4.73]
 [4.75]
 [4.83]
 [4.78]
 [5.01]
 [5.05]
 [4.84]
 [4.87]
 [4.72]
 [4.91]
 [4.92]
 [4.72]
 [5.31]
 [5.56]
 [5.43]
 [5.39]
 [5.53]
 [5.65]
 [5.65]
 [5.4 ]
 [5.5 ]
 [5.36]
 [5.31]
 [5.61]
 [5.57]]


/var/folders/3s/jzblmwmd6bb1z6zxqjy91l8w0000gn/T/ipykernel_47679/1641069171.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x_forecast=np.array(df.drop(['Prediction'],1))[-forecast_out:]


In [113]:
#Print LR model predictions for next 'n' days
lr_prediction=lr.predict(x_forecast)
print(lr_prediction)

#Print SVR model predictions for next 'n' days
svm_prediction=svr_rbf.predict(x_forecast)
print(svm_prediction)

[4.9214663  5.00293532 4.97577898 4.91241419 4.7856846  4.67705925
 4.69516347 4.76758038 4.72231981 4.93051841 4.96672687 4.77663249
 4.80378883 4.66800713 4.83999728 4.8490494  4.66800713 5.20208181
 5.42838464 5.31070717 5.27449871 5.4012283  5.50985366 5.50985366
 5.28355083 5.37407196 5.24734237 5.20208181 5.4736452  5.43743675]
[5.1492988  5.18239293 5.17170638 5.14545094 5.08966747 5.04227574
 5.04994365 5.08160961 5.06166615 5.1531174  5.16806181 5.08563536
 5.09774082 5.03849439 5.11385631 5.11786499 5.03849439 5.24605702
 5.27802488 5.26726042 5.26136818 5.27668636 5.27795404 5.27795404
 5.26295338 5.27466015 5.25616942 5.24605702 5.27873629 5.27831865]
